# Predicting NCAA Men's Basketball Games

This notebook will use the trained spread model and 2 inputed teams to predict the spread of a single game

## Imports

In [180]:
import numpy as np
import pandas as pd
import xgboost as xgb

## Load Models

In [181]:
winner_model = xgb.Booster({'nthread':4}) #init model
winner_model.load_model('../models/winners.model') # load data

spread_model = xgb.Booster({'nthread':4}) #init model
spread_model.load_model('../models/spreads.model') # load data

totals_model = xgb.Booster({'nthread':4}) #init model
totals_model.load_model('../models/totals.model') # load data

## Load Input Data

In [182]:
game_df = pd.read_csv('compact_test.csv')
game_df = game_df.iloc[:, :5]
game_df

,WTeamID,WScore,LTeamID,LScore,OppWinPercentage
0,1211.0,102.0,1.0,90.0,0.833333
1,1211.0,90.0,1.0,67.0,0.500000
2,1211.0,87.0,1.0,82.0,0.692308
3,1211.0,99.0,1.0,88.0,0.833333
4,1211.0,95.0,1.0,57.0,0.076923
5,1211.0,95.0,1.0,78.0,0.076923
6,1211.0,98.0,1.0,75.0,0.777778
7,1211.0,88.0,1.0,58.0,0.272727
8,1211.0,112.0,1.0,67.0,0.666667
9,1211.0,85.0,1.0,62.0,0.615385


In [183]:
wTeamId = 1211
lTeamId = 1339

## Format Data

In [184]:
## Create Dataframe

# Find number of wins for each team in each season
wNumWins = game_df[game_df['WTeamID'] == wTeamId].count()['WTeamID']
lNumWins = game_df[game_df['WTeamID'] == lTeamId].count()['WTeamID']

# Find number of losses for each team
wNumLosses = game_df[game_df['LTeamID'] == wTeamId].count()['LTeamID']
lNumLosses = game_df[game_df['LTeamID'] == lTeamId].count()['LTeamID']

predict_df = pd.DataFrame(data=
                          {
                              'WTeamID': wTeamId,
                              'LTeamID': lTeamId,
                              'WNumWins': wNumWins,
                              'LNumWins': lNumWins,
                              'WNumLosses': wNumLosses,
                              'LNumLosses': lNumLosses
                          }, index=[0])

# Find win percentage for each team
predict_df['WWinPercentage'] = predict_df['WNumWins'] / (predict_df['WNumWins'] + predict_df['WNumLosses'])
predict_df['LWinPercentage'] = predict_df['LNumWins'] / (predict_df['LNumWins'] + predict_df['LNumLosses'])

# Find average win score
predict_df['WAvgWinScore'] = game_df[game_df['WTeamID'] == wTeamId].mean()['WScore']
predict_df['LAvgWinScore'] = game_df[game_df['WTeamID'] == lTeamId].mean()['WScore'] 

# Find average loss score
predict_df['WAvgLossScore'] = game_df[game_df['LTeamID'] == wTeamId].mean()['LScore']
predict_df['LAvgLossScore'] = game_df[game_df['LTeamID'] == lTeamId].mean()['LScore']

# Find Average Win Margin
predict_df['WAvgWinMargin'] = game_df[game_df['WTeamID'] == wTeamId].mean()['WScore'] - game_df[game_df['WTeamID'] == wTeamId].mean()['LScore']
predict_df['LAvgWinMargin'] = game_df[game_df['WTeamID'] == lTeamId].mean()['WScore'] - game_df[game_df['WTeamID'] == lTeamId].mean()['LScore']

# Find Average Loss Margin
predict_df['WAvgLossMargin'] = game_df[game_df['LTeamID'] == wTeamId].mean()['WScore'] - game_df[game_df['LTeamID'] == wTeamId].mean()['LScore']
predict_df['LAvgLossMargin'] = game_df[game_df['LTeamID'] == lTeamId].mean()['WScore'] - game_df[game_df['LTeamID'] == lTeamId].mean()['LScore']

# Find Strength of Schedule
predict_df['WSOS'] = (game_df[game_df['WTeamID'] == wTeamId].mean()['OppWinPercentage'] * wNumWins + game_df[game_df['LTeamID'] == wTeamId].mean()['OppWinPercentage'] * wNumLosses) / (wNumWins + wNumLosses)
predict_df['LSOS'] = (game_df[game_df['WTeamID'] == lTeamId].mean()['OppWinPercentage'] * lNumWins + game_df[game_df['LTeamID'] == lTeamId].mean()['OppWinPercentage'] * lNumLosses) / (lNumWins + lNumLosses)

# Find Adjusted Win Percentage
predict_df['WAdjustedWinPercentage'] = predict_df['WWinPercentage'] * predict_df['WSOS']
predict_df['LAdjustedWinPercentage'] = predict_df['LWinPercentage'] * predict_df['LSOS']

# Subtract relative columns (WNumWins - LNumWins, WAvgWinMargin - LAvgWinMargin, etc.)
predict_df['NumWinsDifference'] = predict_df['WNumWins'] - predict_df['LNumWins']
predict_df['NumLossesDifference'] = predict_df['WNumLosses'] - predict_df['LNumLosses']
predict_df['AvgWinScoreDifference'] = predict_df['WAvgWinScore'] - predict_df['LAvgWinScore']
predict_df['AvgLossScoreDifference'] = predict_df['WAvgLossScore'] - predict_df['LAvgLossScore']
predict_df['AvgWinMarginDifference'] = predict_df['WAvgWinMargin'] - predict_df['LAvgWinMargin']
predict_df['AvgLossMarginDifference'] = predict_df['WAvgLossMargin'] - predict_df['LAvgLossMargin']
predict_df['WinPercentDifference'] = predict_df['WWinPercentage'] - predict_df['LWinPercentage']

# Drop Team IDs
predict_df = predict_df.drop(['WTeamID', 'LTeamID'], axis=1)

# IMPORTANT: The column names must be in the same order as the model!
cols = predict_df.columns.tolist()
predict_df = predict_df[[cols[0]] + [cols[2]] + [cols[4]] + [cols[6]] + [cols[8]] + [cols[10]] + [cols[12]] + [cols[14]] + [cols[16]] + [cols[1]] + [cols[3]] + [cols[5]] + [cols[7]] + [cols[9]] + [cols[11]] + [cols[13]] + [cols[15]] + [cols[17]] + cols[18:]]

predict_df

,WNumWins,WNumLosses,WWinPercentage,WAvgWinScore,WAvgLossScore,WAvgWinMargin,WAvgLossMargin,WSOS,WAdjustedWinPercentage,LNumWins,...,LAvgLossMargin,LSOS,LAdjustedWinPercentage,NumWinsDifference,NumLossesDifference,AvgWinScoreDifference,AvgLossScoreDifference,AvgWinMarginDifference,AvgLossMarginDifference,WinPercentDifference
0,11,0,1.0,94.272727,NaN,22.181818,NaN,NaN,NaN,6,...,22.75,0.397256,0.238354,5,-4,14.272727,NaN,14.515152,NaN,0.4


In [185]:
## Create second DataFrame for point totals
cols = predict_df.columns.tolist()
predict_total_df = predict_df[cols[3:7] + cols[12:16]]

# Subtract relative columns (WNumWins - LNumWins, WAvgWinMargin - LAvgWinMargin, etc.)
predict_total_df['AvgWinScoreSum'] = predict_total_df['WAvgWinScore'] + predict_total_df['LAvgWinScore']
predict_total_df['AvgLossScoreSum'] = predict_total_df['WAvgLossScore'] + predict_total_df['LAvgLossScore']
predict_total_df['WAvgScore'] = (predict_total_df['WAvgWinScore'] + predict_total_df['WAvgLossScore']) / 2
predict_total_df['LAvgScore'] = (predict_total_df['LAvgWinScore'] + predict_total_df['LAvgLossScore']) / 2
predict_total_df['WAvgWinTotalScore'] = game_df[game_df['WTeamID'] == wTeamId].mean()['WScore'] + game_df[game_df['WTeamID'] == wTeamId].mean()['LScore']
predict_total_df['WAvgLossTotalScore'] = game_df[game_df['LTeamID'] == wTeamId].mean()['WScore'] + game_df[game_df['LTeamID'] == wTeamId].mean()['LScore']
predict_total_df['LAvgWinTotalScore'] = game_df[game_df['WTeamID'] == lTeamId].mean()['WScore'] + game_df[game_df['WTeamID'] == lTeamId].mean()['LScore']
predict_total_df['LAvgLossTotalScore'] = game_df[game_df['LTeamID'] == lTeamId].mean()['WScore'] + game_df[game_df['LTeamID'] == lTeamId].mean()['LScore']
predict_total_df['AvgTotalScore'] = (predict_total_df['WAvgWinTotalScore'] + predict_total_df['WAvgLossTotalScore'] + predict_total_df['LAvgWinTotalScore'] + predict_total_df['LAvgLossTotalScore']) / 4

cols = predict_total_df.columns.tolist()
predict_total_df = predict_total_df[cols[:2] + cols[12:14] + cols[2:6] + cols[14:16] + cols[6:12] + [cols[16]]]

predict_total_df

c:\users\mitch mccurry\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\users\mitch mccurry\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
c:\users\mitch mccurry\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

,WAvgWinScore,WAvgLossScore,WAvgWinTotalScore,WAvgLossTotalScore,WAvgWinMargin,WAvgLossMargin,LAvgWinScore,LAvgLossScore,LAvgWinTotalScore,LAvgLossTotalScore,LAvgWinMargin,LAvgLossMargin,AvgWinScoreSum,AvgLossScoreSum,WAvgScore,LAvgScore,AvgTotalScore
0,94.272727,NaN,166.363636,NaN,22.181818,NaN,80.0,61.25,152.333333,145.25,7.666667,22.75,174.272727,NaN,NaN,70.625,NaN


## Run Prediction Models

In [186]:
# Run models
d_test = xgb.DMatrix(predict_df)

pred = winner_model.predict(d_test)
print("Predicted Winner (1 for W team, 0 for L team): ", pred[0])

pred = spread_model.predict(d_test)
print("Predicted Spread: (positive for W team, negative for L team)", pred[0])

d_test2 = xgb.DMatrix(predict_total_df)

pred = totals_model.predict(d_test2)
print("Predicted Total Score: ", pred[0])

Predicted Winner (1 for W team, 0 for L team):  0.8365914
Predicted Spread: (positive for W team, negative for L team) 11.946413
Predicted Total Score:  96.722046
